# Automatic Tumour Segmentation

In [ ]:
import SimpleITK as sitk

from platipy.imaging import ImageVisualiser

from pathlib import Path

%matplotlib notebook

In [ ]:
"""
Step 1 

Get the list of patients
"""

input_dir = Path("/media/robbie/My Passport/Work/3_ResearchProjects/PET-LAB/1_data/PROCESSED/")

patient_id_list = sorted([i.name[4:] for i in input_dir.glob("WES*")])
print(len(patient_id_list))
print(patient_id_list)

In [ ]:
patient_id = "005"

In [ ]:
"""
Step 2

Generate a breast mask

"""

for patient_id in patient_id_list:
    
    print(patient_id)

    img_t1w = sitk.ReadImage( str(input_dir / f"WES_{patient_id}/IMAGES/WES_{patient_id}_TIMEPOINT_1_MRI_T1W_NFS.nii.gz"), sitk.sitkFloat32 )
    img_t2w = sitk.ReadImage( str(input_dir / f"WES_{patient_id}/IMAGES/WES_{patient_id}_TIMEPOINT_1_MRI_T2W.nii.gz"), sitk.sitkFloat32 )
    
    values_t1w = sitk.GetArrayFromImage(img_t1w).flatten()
    values_t2w = sitk.GetArrayFromImage(img_t2w).flatten()
    
    img_t1w_norm = img_t1w/np.percentile(values_t1w, 99.9)
    img_t2w_norm = img_t2w/np.percentile(values_t2w, 99.9)
    
    mask_t1w = img_t1w_norm>0.2
    mask_t1w = sitk.RelabelComponent(sitk.ConnectedComponent(mask_t1w))==1
    
    mask_t2w = img_t2w_norm>0.2
    mask_t2w = sitk.RelabelComponent(sitk.ConnectedComponent(mask_t2w))==1
    
    vis = ImageVisualiser(img_t1w_norm, window=(0,1))
    vis.add_scalar_overlay(mask_t1w)
    fig = vis.show()
    fig.savefig(f"../1_processing/BREAST_MASKS/WES_{patient_id}_T1W.jpeg", dpi=300)
    
    vis = ImageVisualiser(img_t2w_norm, window=(0,1))
    vis.add_scalar_overlay(mask_t2w)
    fig = vis.show()
    fig.savefig(f"../1_processing/BREAST_MASKS/WES_{patient_id}_T2W.jpeg", dpi=300)

In [ ]:
patient_id = "005"

img_t1w = sitk.ReadImage( str(input_dir / f"WES_{patient_id}/IMAGES/WES_{patient_id}_TIMEPOINT_1_MRI_T1W_NFS.nii.gz"), sitk.sitkFloat32 )
img_t2w = sitk.ReadImage( str(input_dir / f"WES_{patient_id}/IMAGES/WES_{patient_id}_TIMEPOINT_1_MRI_T2W.nii.gz"), sitk.sitkFloat32 )

values_t1w = sitk.GetArrayFromImage(img_t1w).flatten()
values_t2w = sitk.GetArrayFromImage(img_t2w).flatten()

img_t1w_norm = img_t1w/np.percentile(values_t1w, 99.9)
img_t2w_norm = img_t2w/np.percentile(values_t2w, 99.9)

mask_t1w = img_t1w_norm>0.2
mask_t1w = sitk.RelabelComponent(sitk.ConnectedComponent(mask_t1w))==1

mask_t2w = img_t2w_norm>0.2
mask_t2w = sitk.RelabelComponent(sitk.ConnectedComponent(mask_t2w))==1

In [ ]:
mask_t1w_filled = sitk.BinaryMorphologicalClosing(mask_t1w, (20,20,2))

In [ ]:
vis = ImageVisualiser(img_t1w_norm, window=(0,1))
vis.add_scalar_overlay(mask_t1w_filled)
fig = vis.show()

In [ ]:
"""
Split into L/R breasts
"""
sag_coords = np.where(sitk.GetArrayFromImage(mask_t1w)==1)[2]
cutoff = int(0.5*(sag_coords.min() + sag_coords.max()))
print(cutoff)

In [ ]:
arr = sitk.GetArrayFromImage(mask_t1w)

In [ ]:
_, _, sag_indices = np.indices(arr.shape)

In [ ]:
arr = sitk.GetArrayFromImage(mask_t1w)
arr[sag_indices>=cutoff] = 0
mask_t1w_rightbreast = sitk.GetImageFromArray(arr)
mask_t1w_rightbreast.CopyInformation(mask_t1w)

In [ ]:
vis = ImageVisualiser(img_t1w_norm, window=(0,1))
vis.add_scalar_overlay(mask_t1w_rightbreast)
fig = vis.show()

In [ ]:
img_dce = sitk.ReadImage( str(input_dir / f"WES_{patient_id}/IMAGES/WES_{patient_id}_TIMEPOINT_1_MRI_T1W_DCE_ACQ_0.nii.gz"), sitk.sitkFloat32 )

In [ ]:
mask_resampled = sitk.Resample(mask_t1w_rightbreast, img_dce, sitk.Transform(), sitk.sitkNearestNeighbor)

In [ ]:
vis = ImageVisualiser(img_dce, window=(0,800))
vis.add_scalar_overlay(mask_resampled)
fig = vis.show()

In [ ]:
"""
Getting shape metrics
"""

lssf = sitk.LabelShapeStatisticsImageFilter()

In [ ]:
lssf.Execute(mask_t1w)

In [ ]:
lssf.GetPerimeter(1)

In [ ]:
"""
Slice-wise connected components
"""

img_t1w_norm_slice = img_t1w_norm[:,:,20]

In [ ]:
test = img_t1w_norm_slice > 0.2
test_mask = sitk.RelabelComponent(sitk.ConnectedComponent(test))==1